In [1]:
import sys
import os
import pandas as pd

# This is a bit of a hack to allow the notebook to import from the services folder
# It adds the parent directory of 'notebooks' (which is your project root) to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from services.data_service import DataService
from services.indicator_service import IndicatorService

print("Libraries and services imported successfully.")

Libraries and services imported successfully.


/Users/aliefchandrawijaya/trading_bot/venv/lib/python3.11/site-packages/pandas_ta/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [2]:
import configparser

# --- LOAD CONFIGURATION ---
config = configparser.ConfigParser()
config.read('../config.ini') 
api_key = config['exchange']['api_key']
api_secret = config['exchange']['api_secret']

# --- Initialize the service WITH the required API keys ---
data_svc = DataService(api_key=api_key, api_secret=api_secret)

# --- Fetch the maximum available historical data ---
# We can go back several years. Let's start from 2018 to get a rich dataset.
START_DATE = "1 Jan, 2018"
historical_df = data_svc.get_all_historical_data(symbol='ETH/USDT', timeframe='4h', start_date=START_DATE)

if historical_df is not None:
    print("\nSuccessfully fetched all historical data.")
    print(f"Total Candles Fetched: {len(historical_df)}")
    print(f"Data Range: from {historical_df.index.min()} to {historical_df.index.max()}")
    display(historical_df.head())

DataService: Binance.US client initialized and connection successful (Read-Only).
DataService: Fetching all historical klines for ETHUSDT since 1 Jan, 2018...
DataService: Downloaded 13005 total klines.
DataService: All historical data fetched and formatted successfully.

Successfully fetched all historical data.
Total Candles Fetched: 13005
Data Range: from 2019-09-23 08:00:00 to 2025-08-30 16:00:00


,open,high,low,close,volume
timestamp,,,,,
2019-09-23 08:00:00,209.55,209.55,209.55,209.55,0.01
2019-09-23 12:00:00,209.55,209.55,209.55,209.55,0.00
2019-09-23 16:00:00,209.55,209.55,209.55,209.55,0.00
2019-09-23 20:00:00,209.55,209.55,209.55,209.55,0.00
2019-09-24 00:00:00,209.55,209.55,209.55,209.55,0.00


In [3]:
indicator_svc = IndicatorService()

# Add all the indicators to our historical data
features_df = indicator_svc.add_all_indicators(historical_df)

if features_df is not None:
    print("Successfully added all indicators.")
    print(f"New data shape: {features_df.shape}")
    print("Columns added:", [col for col in features_df.columns if col not in historical_df.columns])
    print("Data Head with Indicators:")
    display(features_df.head())

IndicatorService: Initialized.
IndicatorService: Calculating and adding a full suite of technical indicators...
IndicatorService: All indicators successfully calculated and added.
Successfully added all indicators.
New data shape: (12780, 30)
Columns added: ['ichimoku_senkou_span_a', 'ichimoku_senkou_span_b', 'ichimoku_tenkan_sen', 'ichimoku_kijun_sen', 'ichimoku_chikou_span', 'EMA_21', 'EMA_50', 'SMA_200', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14', 'ADX_14', 'DMP_14', 'DMN_14', 'SQZ_20_2.0_20_1.5', 'SQZ_ON', 'SQZ_OFF', 'SQZ_NO']
Data Head with Indicators:


,open,high,low,close,volume,ichimoku_senkou_span_a,ichimoku_senkou_span_b,ichimoku_tenkan_sen,ichimoku_kijun_sen,ichimoku_chikou_span,...,BBB_20_2.0,BBP_20_2.0,ATRr_14,ADX_14,DMP_14,DMN_14,SQZ_20_2.0_20_1.5,SQZ_ON,SQZ_OFF,SQZ_NO
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-10-26 12:00:00,181.35,183.25,178.22,180.85,1126.59285,174.0175,178.475,178.640,175.21,182.98,...,23.468057,0.808489,5.361738,39.691356,41.211618,18.255009,21.781667,0,1,0
2019-10-26 16:00:00,181.01,181.37,174.00,176.45,972.85833,174.0175,178.475,179.285,175.21,182.76,...,23.695357,0.682133,5.505185,38.698888,37.270791,21.984749,20.873333,0,1,0
2019-10-26 20:00:00,176.35,179.85,176.35,179.47,332.56844,173.9075,178.475,181.155,175.21,182.02,...,23.387947,0.732997,5.361957,37.777311,35.533049,20.959715,20.858333,0,1,0
2019-10-27 00:00:00,179.57,181.58,176.89,177.50,607.72330,174.0925,178.475,185.490,175.21,180.28,...,22.966756,0.664517,5.313961,37.144614,35.618404,19.638383,19.245000,0,1,0
2019-10-27 04:00:00,177.46,179.62,176.24,178.61,474.52700,173.6275,178.475,185.490,175.21,181.60,...,22.646074,0.673408,5.175820,36.402931,33.956966,19.619369,17.770000,0,1,0


In [4]:
# Create a 'data' directory if it doesn't exist to store our processed datasets
data_dir = '../data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Save the DataFrame to a CSV file
output_path = os.path.join(data_dir, 'eth_usdt_h4_features.csv')
features_df.to_csv(output_path)

print(f"Processed data with features saved successfully to: {output_path}")

Processed data with features saved successfully to: ../data/eth_usdt_h4_features.csv
